# loading required data and libs

In [5]:
import json
import import_ipynb
from importlib import reload
import utils
import inspect
from pathlib import Path
import pylev

import numpy as np
import pandas as pd
from scipy import sparse
import sys
import utils

from sklearn.model_selection import GroupShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer

pd.options.display.width = 180
pd.options.display.max_colwidth = 120

data_dir = Path('data/')
sys.path.append(str('AI4Code'))
%load_ext autoreload
%autoreload 2

importing Jupyter notebook from utils.ipynb


In [ ]:
reload(utils)
from utils import read_train_data, get_df_orders_and_ranks, get_ancestors, count_hastags

In [8]:
df = read_train_data(data_dir, NUM_TRAIN=1000)
df_orders, df_ranks = get_df_orders_and_ranks(df, data_dir)

Train NBs: 100%|██████████| 1000/1000 [00:10<00:00, 98.64it/s]


# applying feature

## Procesisng Bert
https://www.kaggle.com/code/parulpandey/eda-and-preprocessing-for-bert

In [18]:
markdowns = df[df['cell_type'] == 'markdown']
codes = df[df['cell_type'] == 'code']

In [23]:
codes.loc[:, 'source_clean'] = codes['source'].apply(str).apply(lambda x: code_preprocessing(x)).copy().values
markdowns.loc[:, 'source_clean'] = markdowns['source'].apply(str).apply(lambda x: text_preprocessing(x)).copy().values

In [24]:
df = pd.concat([codes, markdowns], ignore_index = True)

In [25]:
df['text_len'] = df['source_clean'].astype(str).apply(len)
df['text_word_count'] = df['source_clean'].apply(lambda x: len(str(x).split()))

In [29]:
from langdetect import detect

In [ ]:
df['language'] = df['source_clean'].apply(lambda x: detect(x) if x else None).copy().values()

In [83]:
sentences = df[df['cell_type']=="markdown"]['source_clean'].values

In [85]:
from sentence_transformers import SentenceTransformer, util

In [86]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/opt/conda/lib/python3.7/site-packages/huggingface_hub/file_download.py:563: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  FutureWarning,


In [87]:
paraphrases = util.paraphrase_mining(
    model,
    list(set(sentences))[:4000], 
    show_progress_bar=True,
    max_pairs=50,
)

In [88]:
import textwrap

In [2]:
for paraphrase in paraphrases[10:30]:
    score, i, j = paraphrase
    print(f"Score {score}")
    print('/n'.join(textwrap.wrap(list(set(sentences))[i], width=70)))
    print(">"*10)
    print('/n'.join(textwrap.wrap(list(set(sentences))[j], width=70)))
    print("-"*20)
    # print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], score))

NameError: name 'paraphrases' is not defined

# Smart splitting via group shuffling

In [6]:
NVALID = 0.1  # size of validation set

splitter = GroupShuffleSplit(n_splits=1, test_size=NVALID, random_state=0)

ids = df.index.unique('id')  # get all the unique ids
ancestors = get_ancestors(data_dir, ids)  # find ancestor by id if it exists
# split the ids using groups. This way the same group/notebooks will be in the test or in the training
ids_train, ids_valid = next(splitter.split(ids, groups=ancestors)) 
ids_train, ids_valid = ids[ids_train], ids[ids_valid]

df_train = df.loc[ids_train, :]
df_valid = df.loc[ids_valid, :]

print(f"Shape of train: {df_train.shape[0]}; validation: {df_valid.shape[0]}")

Shape of train: 126414; validation: 14849


## preparing training set

In [7]:
def mark_each_cell_with_its_position(current_X, full_df):
    """
    marking each cell with its number if its code, for markdown zero. 
    We are doing it to help the model learn the correct order in lines?
    """
    old_shape = current_X.shape
    current_X = sparse.hstack((
        current_X,
        np.where(
            full_df['cell_type'] == 'code',
            full_df.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1,
            0,
        ).reshape(-1, 1)
    ))
    new_shape = current_X.shape
    print(f"Shape change {old_shape} -> {new_shape}")
    return current_X

In [8]:
tfidf = TfidfVectorizer(min_df=0.01, max_features=170, stop_words='english')  # idf(t) = log [ n / df(t) ] + 1, where df(t) – number of time term is used
def convert_to_TfidfVector(df):
    print("Converting with Tfid vectorizer")
    return tfidf.fit_transform(df.astype(str))

In [9]:
def add_custom_column_to_sparse(current_X, full_df, column):
    print(f"Added {column} to the dataframe")
    return sparse.hstack((
        current_X,
        full_df['hash_count'].values.reshape(-1, 1)
    ))

In [10]:
X_train = convert_to_TfidfVector(df_train['source'])
X_train = mark_each_cell_with_its_position(X_train, df_train)
X_train = add_custom_column_to_sparse(X_train, df_train, "hash_count")

Converting with Tfid vectorizer
Shape change (126414, 170) -> (126414, 171)
Added hash_count to the dataframe


In [11]:
y_train = df_ranks.loc[ids_train].to_numpy()  # get all required train results
groups = df_ranks.loc[ids_train].groupby('id').size().to_numpy() # Number of cells in each notebook. will later be used to help xgboost make a ranking

In [12]:
from xgboost import XGBRanker

model = XGBRanker(
    min_child_weight=10,
    subsample=0.5,
    tree_method='hist',
)
model.fit(X_train, y_train, group=groups)

XGBRanker(base_score=0.5, booster='gbtree', callbacks=None, colsample_bylevel=1,
          colsample_bynode=1, colsample_bytree=1, early_stopping_rounds=None,
          enable_categorical=False, eval_metric=None, gamma=0, gpu_id=-1,
          grow_policy='depthwise', importance_type=None,
          interaction_constraints='', learning_rate=0.300000012, max_bin=256,
          max_cat_to_onehot=4, max_delta_step=0, max_depth=6, max_leaves=0,
          min_child_weight=10, missing=nan, monotone_constraints='()',
          n_estimators=100, n_jobs=0, num_parallel_tree=1, predictor='auto',
          random_state=0, reg_alpha=0, reg_lambda=1, ...)

## validation

In [13]:
X_valid = convert_to_TfidfVector(df_valid['source'])
X_valid = mark_each_cell_with_its_position(X_valid, df_valid)
X_valid = add_custom_column_to_sparse(X_valid, df_valid, "hash_count")

Converting with Tfid vectorizer
Shape change (14849, 170) -> (14849, 171)
Added hash_count to the dataframe


In [16]:
y_valid = df_orders.loc[ids_valid]

In [17]:
y_pred = pd.DataFrame({'rank': model.predict(X_valid)}, index=df_valid.index)
y_pred = (
    y_pred
    .sort_values(['id', 'rank'])  # Sort the cells in each notebook by their rank.
                                  # The cell_ids are now in the order the model predicted.
    .reset_index('cell_id')  # Convert the cell_id index into a column.
    .groupby('id')['cell_id'].apply(list)  # Group the cell_ids for each notebook into a list.
)

# Metrics

In [20]:
from bisect import bisect


def count_inversions(a):
    inversions = 0
    sorted_so_far = []
    for i, u in enumerate(a):
        j = bisect(sorted_so_far, u)
        inversions += i - j
        sorted_so_far.insert(j, u)
    return inversions


def kendall_tau(ground_truth, predictions):
    total_inversions = 0
    total_2max = 0  # twice the maximum possible inversions across all instances
    for gt, pred in zip(ground_truth, predictions):
        ranks = [gt.index(x) for x in pred]  # rank predicted order in terms of ground truth
        total_inversions += count_inversions(ranks)
        n = len(gt)
        total_2max += n * (n - 1)
    return 1 - 4 * total_inversions / total_2max


In [21]:
y_dummy = df_valid.reset_index('cell_id').groupby('id')['cell_id'].apply(list)
kendall_tau(y_valid, y_dummy)

0.392139890255764

In [23]:
kendall_tau(y_valid, y_pred)

0.506034709520343